In [1]:
# 传统神经网络存在的问题:
# 1.权值太多
# 2.速度慢
# Tip:
#     数据量小,网络复杂---过拟合
#     数据量大,网络简单---欠拟合
# Tip:
#     权值 : 数据量: 1:5~30倍

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
# 一、载入数据
mnist=input_data.read_data_sets('Tensorflow_data',one_hot=True)

batch_size=100
    # 定义每个批次的大小--训练模型的时候一批次100个
n_batch=mnist.train.num_examples // batch_size
    # n_batch---一共有多少个批次(计算数据集总数量 // 批次大小)
    # // 整除   

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting Tensorflow_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting Tensorflow_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting Tensorflow_data\t10k-images-idx3-ubyte.gz
Extracting Tensorflow_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
# 二、初始化权值
def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
        # 生成一个截断的正太分布
    return tf.Variable(initial)

# 初始化偏置
def bias_variale(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

In [5]:
# 三、卷积层调用
# 使用的是Tensorflow里的一个库
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
# conv2d---二维的卷积操作
# x [批次,长,宽,黑白/彩色--1/3]

In [6]:
# 四、池化层调用
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
# kszie[1,x,y,1] 只有中间两个自己调节

In [7]:
# 五、 卷积,池化
# 定义变量
x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.float32,[None,10])

In [8]:
# 改变x的格式为4D的向量 [批次,长,宽,黑白/彩色--1/3]
x_image=tf.reshape(x,[-1,28,28,1])

In [9]:
# 初始化第一个卷积层的权值和偏置值
W_conv1=weight_variable([5,5,1,32]) # 5×5的采样窗口,32个卷积核从1个平面抽取特征
b_conv1=bias_variale([32]) # 每一个卷积核一个偏置值

In [10]:
# 第一层卷积,池化
h_conv1=tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1) # 卷积操作
h_pool1=max_pool_2x2(h_conv1) # 池化操作


In [11]:
# 初始化第二个卷积层的权值和偏置值
W_conv2=weight_variable([5,5,32,64]) # 5×5的采样窗口,32个卷积核从1个平面抽取特征
b_conv2=bias_variale([64]) # 每一个卷积核一个偏置值

In [12]:
# 第二层卷积,池化
h_conv2=tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2) # 卷积操作
h_pool2=max_pool_2x2(h_conv2) # 池化操作

In [13]:
# 28*28的图片
# 第一次卷积后还是28*28,
# 第一次池化后变为14*14
# 第二次卷积后14*14
# 第二次池化后7*7
# 得到64张7*7 

In [14]:
# 七、初始化第一个全连接层的权值
W_fc1=weight_variable([7*7*64,1024]) # 上一层有7*7*64个神经元,全连接层有1024个神经元
b_fc1=bias_variale([1024]) # 1024个节点

In [15]:
# 把第二层池化层的输出扁平化为1维
h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])

In [16]:
# 第一个全连接层的输出
h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

# 概率表示
keep_prob=tf.placeholder(tf.float32)
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)

In [17]:
# 初始化第二个全连接层
W_fc2=weight_variable([1024,10])
b_fc2=bias_variale([10])

# 计算输出
prediction=tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

In [18]:
# 交叉熵代价函数
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
# 使用AdamOptimizer进行优化---优化器
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# 结果存放在一个布尔列表中
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [ ]:
# 求准确率
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(21):
        for batch in range(n_batch):
            xx,yy=mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:xx,y:yy,keep_prob:0.7})
        acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print("值:"+str(i)+",测试集准确率:"+str(acc))
            
            
    












值:0,测试集准确率:0.859
值:1,测试集准确率:0.874
值:2,测试集准确率:0.9696
值:3,测试集准确率:0.98
值:4,测试集准确率:0.9853
